1) Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

2) Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

3) Повторить п.2, но используя уже не медиану, а max

4) (опциональное) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.

5) Сформировать на выходе единую таблицу, сравнивающую качество 4 разных методов получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

6) Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [1]:
import pandas as pd
import numpy as np

import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.corpus import stopwords
from razdel import tokenize
import pymorphy2
from gensim.corpora.dictionary import Dictionary

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dolas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [4]:
news = pd.read_csv("materials.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [5]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [6]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [7]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [8]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-7-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 21.2 s


In [9]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 2min 37s


In [10]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [11]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 22.7 s


In [12]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [13]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(5, 0.028580407),
 (7, 0.27453008),
 (9, 0.43868056),
 (14, 0.16412441),
 (19, 0.050772034),
 (22, 0.026365167)]

In [14]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: год который это исследование первый время весь
topic_1: тело смерть женщина врач трагедия выдать реакция
topic_2: экипаж гостиница порт рим молдавия эндрю перо
topic_3: украина украинский киев сигнал вуз это человек
topic_4: дело сотрудник суд человек полиция уголовный орган
topic_5: женщина мужчина египет летний высота бизнесмен тело
topic_6: потенциально шутка присудить скотт фантазия стерлинг аннулировать
topic_7: это который мочь год человек свой всё
topic_8: год это который nn млн рубль проект
topic_9: достигать германия польша игра сон италия тепло
topic_10: ракета км километр перевод пилотировать горизонт секунда
topic_11: остров япония карта сектор мэй берег выдавать
topic_12: дождь офицер водитель скорость инвестировать корь многолетний
topic_13: рак место фильм nn кость год фрагмент
topic_14: конструкция лауреат рт добраться кг балл столетие
topic_15: обнаружить это президент путин данные источник государство
topic_16: северный земля запуск умереть следствие океан уп

In [15]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [16]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.014791,...,0.0,0.0,0.000000,0.849105,0.000000,0.0,0.0,0.000000,0.0,0.000000
1,4896,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.542823,0.0,0.0,0.000000,0.0,0.000000
2,4897,0.000000,0.0,0.0,0.0,0.0,0.02858,0.0,0.274536,0.000000,...,0.0,0.0,0.000000,0.000000,0.050772,0.0,0.0,0.026365,0.0,0.000000
3,4898,0.671976,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.072674,0.000000,0.0,0.0,0.000000,0.0,0.032717
4,4899,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,...,0.0,0.0,0.194802,0.649618,0.000000,0.0,0.0,0.000000,0.0,0.000000


In [17]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [18]:
user_articles_list = users['articles'].iloc[33]

In [19]:
def get_user_embedding(user_articles_list, mode):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    if mode == 'mean':
        user_vector = np.mean(user_vector, 0)
    if mode == 'median':
        user_vector = np.median(user_vector, 0)
    if mode == 'max':
        user_vector = np.amax(user_vector, 0)
    if mode == 'idf_mean':
        return np.nan
    return user_vector

In [20]:
# get_user_embedding(user_articles_list)

In [21]:
# user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
# user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
# user_embeddings['uid'] = users['uid'].values
# user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
# user_embeddings.head(3)

In [22]:
# target = pd.read_csv("users_churn.csv")
# target.head(3)

In [23]:
# X = pd.merge(user_embeddings, target, 'left')
# X.head(3)

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [31]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [26]:
# #разделим данные на train/test
# X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
#                                                     X['churn'], random_state=0)

In [27]:
# logreg = LogisticRegression()
# #обучим 
# logreg.fit(X_train, y_train)

In [28]:
# #наши прогнозы для тестовой выборки
# preds = logreg.predict_proba(X_test)[:, 1]
# preds[:10]

In [29]:
# precision, recall, thresholds = precision_recall_curve(y_test, preds)
# fscore = (2 * precision * recall) / (precision + recall)
# # locate the index of the largest f score
# ix = np.argmax(fscore)
# print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
#                                                                         fscore[ix],
#                                                                         precision[ix],
#                                                                         recall[ix]))

In [34]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [35]:
# #мы уже нашли ранее "оптимальный" порог, когда максимизировали f_score
# font = {'size' : 15}

# plt.rc('font', **font)

# cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])
# plt.figure(figsize=(10, 8))
# plot_confusion_matrix(cnf_matrix, classes=['Non-Churn', 'churn'],
#                       title='Confusion matrix')
# plt.savefig("conf_matrix.png")
# plt.show()

In [36]:
# roc_auc_score(y_test, preds)

         roc_auc| precision| recall| f_score
         ___________________________________
mean    |       |          |       |        |
median  |       |          |       |        |
max     |       |          |       |        |
idf_mean|       |          |       |        |

In [37]:
def task_2(mode, user_articles_list, doc_dict, users):
    output_data = []
    if mode == 'mean':
        
        user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, mode), 1)])
        user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
        user_embeddings['uid'] = users['uid'].values
        user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
        
        target = pd.read_csv("users_churn.csv")
        X = pd.merge(user_embeddings, target, 'left')
        
        X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)
        logreg = LogisticRegression()
        logreg.fit(X_train, y_train)
        preds = logreg.predict_proba(X_test)[:, 1]
        precision, recall, thresholds = precision_recall_curve(y_test, preds)
        fscore = (2 * precision * recall) / (precision + recall)
        # locate the index of the largest f score
        ix = np.argmax(fscore)
        print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, roc_auc_score=%.6f'  % (thresholds[ix], 
                                                                              fscore[ix],
                                                                                precision[ix],
                                                                                recall[ix], roc_auc_score(y_test, preds)))
        output_data = [fscore[ix],precision[ix],recall[ix], roc_auc_score(y_test, preds)]
        return output_data
    
    if mode == 'median':
        
        
        user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, mode), 1)])
        user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
        user_embeddings['uid'] = users['uid'].values
        user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
        
        target = pd.read_csv("users_churn.csv")
        X = pd.merge(user_embeddings, target, 'left')
        
        X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)
        logreg = LogisticRegression()
        logreg.fit(X_train, y_train)
        preds = logreg.predict_proba(X_test)[:, 1]
        precision, recall, thresholds = precision_recall_curve(y_test, preds)
        fscore = (2 * precision * recall) / (precision + recall)
        # locate the index of the largest f score
        ix = np.argmax(fscore)
        print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, roc_auc_score=%.6f'  % (thresholds[ix], 
                                                                                fscore[ix],
                                                                                precision[ix],
                                                                                recall[ix], roc_auc_score(y_test, preds)))
        output_data = [fscore[ix],precision[ix],recall[ix], roc_auc_score(y_test, preds)]
        return output_data
    
    if mode == 'max':
        
        
        user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, mode), 1)])
        user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
        user_embeddings['uid'] = users['uid'].values
        user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
        
        target = pd.read_csv("users_churn.csv")
        X = pd.merge(user_embeddings, target, 'left')
        
        X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)
        logreg = LogisticRegression()
        logreg.fit(X_train, y_train)
        preds = logreg.predict_proba(X_test)[:, 1]
        precision, recall, thresholds = precision_recall_curve(y_test, preds)
        fscore = (2 * precision * recall) / (precision + recall)
        # locate the index of the largest f score
        ix = np.argmax(fscore)
        print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, roc_auc_score=%.6f'  % (thresholds[ix], 
                                                                                fscore[ix],
                                                                                precision[ix],
                                                                                recall[ix], roc_auc_score(y_test, preds)))
        output_data = [fscore[ix],precision[ix],recall[ix], roc_auc_score(y_test, preds)]
        return output_data
    if mode == 'idf_mean':
        pass
    return output_data

In [40]:
tests = ['mean', 'median', 'max', 'idf_mean']
tests_result = []
tests_result_label = ['F-Score', 'Precision', 'Recall', 'roc_auc_score']
for each in tests:
    tests_result.append(task_2(each, user_articles_list, doc_dict, users))

Best Threshold=0.295787, F-Score=0.693, Precision=0.700, Recall=0.686, roc_auc_score=0.946118
Best Threshold=0.260213, F-Score=0.718, Precision=0.638, Recall=0.820, roc_auc_score=0.963033
Best Threshold=0.358469, F-Score=0.809, Precision=0.806, Recall=0.812, roc_auc_score=0.978406


In [41]:
df = pd.DataFrame(tests_result, columns=tests_result_label, index=tests)
df

,F-Score,Precision,Recall,roc_auc_score
mean,0.692784,0.700000,0.685714,0.946118
median,0.717857,0.638095,0.820408,0.963033
max,0.808943,0.805668,0.812245,0.978406
idf_mean,NaN,NaN,NaN,NaN


разница из-за того, что мы смотрим на вектор в трёх (четырёх) вариантах, один медиана, один средний и один максимальный
где в среднем пользователь смотрел на эти темы
или где располагается медианное значение среди тем
и максимальное больше всех, потому что пользователь смотрел больше всего этих тем (максимальное среди тем)